In [1]:
import pandas as pd 
import numpy as np 
import math as math
import joblib
from joblib import dump
import os
import time

In [3]:
import pandas as pd 
print(np.__version__)

1.24.4


## LOAD DATA

In [2]:
def load_Dataset(baseFile, fold_Number): 
    rnmColData = ['user_id', 'item_id', 'rating', 'timestamp']
    base_File = f"{baseFile}/u{fold_Number}.base"
    test_File = f"{baseFile}/u{fold_Number}.test"
    base_Data = pd.read_csv(base_File, sep="\t", header=None, names=rnmColData)
    test_Data = pd.read_csv(test_File, sep="\t", header=None, names=rnmColData)
    base_Data = base_Data.drop(columns=["timestamp"])
    test_Data = test_Data.drop(columns=["timestamp"])
    return base_Data, test_Data

In [5]:
call_base = "ml-100k"
basedata, testdata = load_Dataset(call_base, 1)
basedata


,user_id,item_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3
...,...,...,...
79995,943,1067,2
79996,943,1074,4
79997,943,1188,3
79998,943,1228,3


In [6]:
basedata

,user_id,item_id,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3
...,...,...,...
79995,943,1067,2
79996,943,1074,4
79997,943,1188,3
79998,943,1228,3


In [5]:
testdata

,user_id,item_id,rating
0,1,6,5
1,1,10,3
2,1,12,5
3,1,14,5
4,1,17,3
...,...,...,...
19995,458,648,4
19996,458,1101,4
19997,459,934,3
19998,460,10,3


In [7]:
rnmColData = ['user_id', 'item_id', 'rating', 'timestamp']
readData = pd.read_csv("ml-100k/u.data", sep="\t", header=None, names=rnmColData)

In [3]:
def ConvertRatingMatriks(ratingData):
    # membuat container untuk rating matriks dengan ukuran user x item
    matriks_rating = pd.DataFrame(np.zeros((943, 1682)), columns = list(range(1, 1683)), index = list(range(1, 944)))
    # merubah data frame ke dalam bentuk matriks rating pivot
    convertMatriksRating = ratingData.pivot_table(index = 'user_id', columns = 'item_id', values='rating')
    # mengisi matriks rating NaN dengan 0
    convertMatriksRating = convertMatriksRating.fillna(0)
    # updating matriks rating dengan matriks rating yang sudah di pivot
    matriks_rating.update(convertMatriksRating)
    # mengembalikan matriks rating
    return matriks_rating

In [44]:
# def ConvertRatingMatriks2(ratingData):
#     # membuat container untuk rating matriks dengan ukuran user x item
#     matriks_rating = pd.DataFrame(np.zeros((943, 1682)), columns = list(range(1, 1683)), index = list(range(1, 944)))
#     # merubah data frame ke dalam bentuk matriks rating pivot
#     convertMatriksRating = ratingData.pivot_table(index = 'user_id', columns = 'item_id', values='rating')
#     # mengisi matriks rating NaN dengan 0
#     matriks_rating = convertMatriksRating.fillna(0)
#     # updating matriks rating dengan matriks rating yang sudah di pivot
#     matriks_rating.update(convertMatriksRating)
#     # mengembalikan matriks rating
#     return matriks_rating

## CONVERT RATING MATRIKS

## BASE MODEL

In [6]:
rating_Matriks = ConvertRatingMatriks(basedata)
print(rating_Matriks.shape)
# rating_Matriks

# dataset 
allItem = rating_Matriks.columns
print(f"all item {len(allItem)}")
itemRated = set(rating_Matriks.columns[rating_Matriks.sum(axis=0) > 0])
print(f"item rated {len(itemRated)}")

# item yang tidak pernah di rating
itemNotRated = set(allItem) - itemRated
print(f"item not rated {len(itemNotRated)}")

(943, 1682)
all item 1682
item rated 1650
item not rated 32


In [8]:
userRatedCounts = rating_Matriks.astype(bool).sum(axis=0)
print(userRatedCounts.describe())


count    1682.000000
mean       47.562426
std        64.753079
min         0.000000
25%         5.000000
50%        21.000000
75%        63.000000
max       484.000000
dtype: float64


In [9]:
rating_Matriks

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
jumlahRating = np.count_nonzero(rating_Matriks)
jumlahRating

80000

### MEAN 

In [10]:
def MeanRating(RatingMatriks, jenis="user-based"):
    if jenis == "user-based":
        # menghitung berapa banyak rating yang ada pada setiap user(baris)
        Id_axis = 1
        # index baris user dari rating matriks
        index = RatingMatriks.index
    elif jenis == "item-based":
        # menghitung berapa banyak rating yang ada pada setiap item(kolom)
        Id_axis = 0
        # index kolom item dari rating matriks
        index = RatingMatriks.columns
    # menghitung mean rating dari setiap user/item
    # jumlah rating yang ada pada setiap user/item yang bukan 0
    Pembilang = RatingMatriks.sum(axis=Id_axis)
    # user 1
    # print(f"Pembilang user: {Pembilang.loc[941]}")
    # type data Pembilang
    # print(f"Pembilang user : {type(Pembilang.loc[941])}")
    # user 1 pada  item 1 yang telah diberikan rating ada pada setiap user/item yang bukan 0
    Penyebut = np.count_nonzero(RatingMatriks, axis=Id_axis)
    # print(f"Penyebut user : {Penyebut[RatingMatriks.index.get_loc(941)]}")
    # cek jika Penyebut == 0, maka mean rating = 0, jika tidak hitung mean rating
    calculateMeanRating = np.where(Penyebut == 0, 0, Pembilang / Penyebut)

    # mengembalikan mean rating dalam bentuk data frame
    MeanRatingDF = pd.DataFrame(calculateMeanRating, index=index, columns=["meanRating"])
    return MeanRatingDF

#### USER

In [11]:
# vektor, karena mean rating hanya satu kolom, ukuran 943 x 1
cal_MeanRatingUser = MeanRating(rating_Matriks, jenis="user-based")
cal_MeanRatingUser

,meanRating
1,3.681481
2,3.800000
3,3.000000
4,4.357143
5,2.956044
...,...
939,4.265306
940,3.457944
941,4.045455
942,4.265823


#### ITEM

In [12]:
cal_MeanRatingItem = MeanRating(rating_Matriks, jenis="item-based")
print(cal_MeanRatingItem.shape)
cal_MeanRatingItem

(1682, 1)


,meanRating
1,3.892950
2,3.180952
3,3.000000
4,3.526316
5,3.304348
...,...
1678,1.000000
1679,3.000000
1680,2.000000
1681,3.000000


### MEAN-CENTERED

In [ ]:
# user based
# print(f'npRatingMatriks : {NpRatingMatriks[RatingMatriks.index.get_loc(941),:]}')
# print(f'npMean : {npMeanRating[RatingMatriks.index.get_loc(941),:]}')
# print(f'MeanCentered : {MeanCentered[RatingMatriks.index.get_loc(941),:]}')

# item based
# print(f'npRatingMatriks : {NpRatingMatriks[:,RatingMatriks.columns.get_loc(1)]}')
# print(f'npMean : {npMeanRating[0,RatingMatriks.columns.get_loc(1)]}')
# print(f'MeanCentered : {MeanCentered[:,RatingMatriks.columns.get_loc(1)]}')

In [13]:
def MeanCenteredRating(RatingMatriks, meanRating, jenis="user-based"):
    # matriks rating jadi numpy array
    NpRatingMatriks = np.array(RatingMatriks)
    # print(f"ratings matriks : {RatingMatriks.shape}, np rating matriks : {NpRatingMatriks.shape}")

    # print(f'npRatingMatriks : {NpRatingMatriks[:,RatingMatriks.columns.get_loc(1)]}')
    if jenis == "user-based":
        # Reshape meanRating menjadi (jumlah_user, 1), kolom
        npMeanRating = np.array(meanRating).reshape(-1, 1)  # (943, 1)
    elif jenis == "item-based":
        # Reshape meanRating menjadi (1, jumlah_item), baris
        npMeanRating = np.array(meanRating).reshape(1, -1)  # (1, 1650)
    # perhitungan rating setiap user dikurangi mean rating user, yang bukan 0
    MeanCentered = np.where(NpRatingMatriks != 0, NpRatingMatriks - npMeanRating, 0)
    # print(f'npRatingMatriks : {NpRatingMatriks[RatingMatriks.index.get_loc(941),:]}')
    # print(f'npMean : {npMeanRating[RatingMatriks.index.get_loc(941),:]}')
    # sudahRated = np.count_nonzero(NpRatingMatriks[RatingMatriks.index.get_loc(941),:])
    # nonRated = len(NpRatingMatriks[RatingMatriks.index.get_loc(941),:])
    # belum_dinilai = RatingMatriks.loc[941][RatingMatriks.loc[941] == 0].index.tolist()

    # print(f'sudahRated : {sudahRated}')
    # print(f'nonRated : {len(belum_dinilai)}')
    # print(f'MeanCentered : {MeanCentered[RatingMatriks.index.get_loc(941),:]}')

    # Mengembalikan dalam bentuk DataFrame
    dfMeanCentered = pd.DataFrame(MeanCentered, index=RatingMatriks.index, columns=RatingMatriks.columns)
    return dfMeanCentered

#### USER

In [14]:
# matriks
cal_MeanCenteredUser = MeanCenteredRating(rating_Matriks, cal_MeanRatingUser, jenis="user-based")
cal_MeanCenteredUser

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.318519,-0.681481,0.318519,-0.681481,-0.681481,0.0,0.318519,-2.681481,1.318519,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.200000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,-1.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.734694,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.000000,0.000000,0.000000,-1.457944,0.000000,0.0,0.542056,1.542056,-0.457944,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.954545,0.000000,0.000000,0.000000,0.000000,0.0,-0.045455,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### ITEM

In [15]:
cal_MeanCenteredItem = MeanCenteredRating(rating_Matriks, cal_MeanRatingItem, jenis="item-based")
print(f"mean cen {cal_MeanCenteredItem.shape}")
cal_MeanCenteredItem

mean cen (943, 1682)


,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.10705,-0.180952,1.0,-0.526316,-0.304348,0.0,0.201954,-2.99422,1.166667,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.10705,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,-1.876712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,1.166667,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.00000,0.000000,0.0,-1.526316,0.000000,0.0,0.201954,1.00578,-0.833333,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,1.10705,0.000000,0.0,0.000000,0.000000,0.0,0.201954,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## SIMILARITY

In [ ]:
# print(f"item rated 1", len(itemRated1))
# print(f"item rated ", itemRated1)
# itemNotRated1 = set(RatingMatriks.columns[RatingMatriks.iloc[index1, :] == 0])
# print(f"item not rated {itemNotRated1}") 

In [20]:
def SimilarityJaccard(RatingMatriks, index1, index2, jenis="user-based"):
    if jenis == "user-based":
        #mengambil seluru rating dari user pada baris index1 dan index2
        #memili nama item yang sudah di rating oleh user >0 
        itemRated1 = set(RatingMatriks.columns[RatingMatriks.iloc[index1, :] != 0])
        # print(f"item rated ", itemRated1)
        itemRated2 = set(RatingMatriks.columns[RatingMatriks.iloc[index2, :] != 0])
    elif jenis == "item-based":
        #mengambil seluru rating untuk item pada kolom index1 dan index2
        #memili nama user yang sudah di rating oleh item >0
        itemRated1 = set(RatingMatriks.index[RatingMatriks.iloc[:, index1] != 0])
        itemRated2 = set(RatingMatriks.index[RatingMatriks.iloc[:, index2] != 0])

    # jumlah item(len)
    intersection = len(itemRated1.intersection(itemRated2))
    union = len(itemRated1.union(itemRated2))
    if union != 0:
        rumusJaccardSim = intersection / union
    else:
        rumusJaccardSim = 0
    return rumusJaccardSim

In [38]:
calSimDuaJaccard = SimilarityJaccard(rating_Matriks, 0, 895, jenis="user-based")
calSimDuaJaccard

0.1585081585081585

In [36]:
def SimilarityAllJaccard(RatingMatriks, jenis="user-based"):
    # jika jenis user-based
    if jenis == "user-based" : 
        # mendapatkan jumlah user(baris) dari rating matriks
        jumlahUser = RatingMatriks.shape[0]
        # membuat container untuk matriks similarity dengan ukuran user x user
        matriksSimilarity = np.zeros((jumlahUser, jumlahUser))
        # mengisi diagonal matriks similarity dengan 1
        np.fill_diagonal(matriksSimilarity, 1)
        # i loop setiap user baris atas
        for i in range (jumlahUser):
            # j loop setiap user lain, mulai dari user ke i+1 agar tidak menghitung user yang sama
            for j in range(i+1, jumlahUser):
                # menghitung jaccard antara user i dan user j
                rumusSimJaccardUser = SimilarityJaccard(RatingMatriks, i, j, jenis="user-based")
                # menyimpan hasil sim dari user i dan j
                matriksSimilarity[i][j] = rumusSimJaccardUser
                matriksSimilarity[j][i] = rumusSimJaccardUser
        # mengembalikan hasil similarity dalam bentuk data frame
        hasilSimJaccard = pd.DataFrame(matriksSimilarity, index=RatingMatriks.index, columns=RatingMatriks.index)
    # jika jenis item-based
    elif jenis == "item-based" :
        # mendapatkan jumlah item(kolom) dari rating matriks 
        jumlahItem = RatingMatriks.shape[1]
        # membuat container untuk matriks similarity dengan ukuran item x item
        matriksSimilarity = np.zeros((jumlahItem, jumlahItem))
        # mengisi diagonal matriks similarity dengan 1
        np.fill_diagonal(matriksSimilarity, 1)
        # i loop setiap item
        for i in range (jumlahItem):
            # j loop setiap item lain, mulai dari item ke i+1 agar tidak menghitung item yang sama
            for j in range(i+1, jumlahItem):
                # menghitung jaccard antara item i dan item j
                rumusSimJaccardItem = SimilarityJaccard(RatingMatriks, i, j, jenis="item-based")
                # menyimpan hasil sim dari item i dan j
                matriksSimilarity[i][j] = rumusSimJaccardItem
                matriksSimilarity[j][i] = rumusSimJaccardItem
        # mengembalikan hasil similarity dalam bentuk data frame
        hasilSimJaccard = pd.DataFrame(matriksSimilarity, index=RatingMatriks.columns, columns=RatingMatriks.columns)
    return hasilSimJaccard

In [265]:
cal_SimiAll = SimilarityAllJaccard(rating_Matriks, jenis="user-based")
cal_SimiAll

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.035503,0.018750,0.013605,0.102439,0.155660,0.101266,0.031250,0.013793,0.106280,...,0.136029,0.041916,0.099206,0.060606,0.080000,0.039548,0.115207,0.046667,0.070000,0.130597
2,0.035503,1.000000,0.046154,0.038462,0.007692,0.094891,0.032653,0.014493,0.061224,0.038760,...,0.033816,0.161765,0.144654,0.230769,0.138462,0.098765,0.088889,0.087719,0.062500,0.034826
3,0.018750,0.046154,1.000000,0.076923,0.008475,0.045455,0.008368,0.054545,0.000000,0.025210,...,0.015075,0.015152,0.055901,0.062500,0.038168,0.026667,0.071429,0.041667,0.038835,0.005128
4,0.013605,0.038462,0.076923,1.000000,0.009615,0.000000,0.013393,0.047619,0.000000,0.009346,...,0.005348,0.019231,0.040000,0.058824,0.033898,0.016129,0.043103,0.090909,0.056818,0.011111
5,0.102439,0.007692,0.008475,0.009615,1.000000,0.086486,0.105455,0.052174,0.019802,0.051136,...,0.147186,0.031746,0.040179,0.015504,0.064171,0.029412,0.093923,0.036697,0.075949,0.146018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.039548,0.098765,0.026667,0.016129,0.029412,0.052980,0.031496,0.012821,0.000000,0.014184,...,0.032407,0.257143,0.136905,0.126582,0.226562,1.000000,0.033113,0.092308,0.024000,0.058537
940,0.115207,0.088889,0.071429,0.043103,0.093923,0.160428,0.134752,0.045802,0.025862,0.161850,...,0.190678,0.050360,0.116592,0.097015,0.091371,0.033113,1.000000,0.057377,0.141104,0.113360
941,0.046667,0.087719,0.041667,0.090909,0.036697,0.039370,0.008584,0.083333,0.062500,0.008696,...,0.015544,0.109091,0.108108,0.127273,0.120690,0.092308,0.057377,1.000000,0.041237,0.027027


In [24]:
cal_SimiAllItemBase = SimilarityAllJaccard(rating_Matriks, jenis="item-based")
cal_SimiAllItemBase

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.000000,0.178744,0.150754,0.236607,0.091787,0.022843,0.416838,0.246637,0.302277,0.106796,...,0.002611,0.0,0.000000,0.000000,0.002611,0.0,0.0,0.0,0.002611,0.002611
2,0.178744,1.000000,0.139241,0.301887,0.175676,0.041667,0.187320,0.168067,0.103774,0.072289,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.009524,0.009524
3,0.150754,0.139241,1.000000,0.160377,0.090909,0.043956,0.147147,0.127273,0.150538,0.112782,...,0.000000,0.0,0.000000,0.000000,0.013333,0.0,0.0,0.0,0.000000,0.013333
4,0.236607,0.301887,0.160377,1.000000,0.170732,0.043716,0.257895,0.269373,0.208696,0.104072,...,0.000000,0.0,0.005848,0.005848,0.005848,0.0,0.0,0.0,0.005848,0.005848
5,0.091787,0.175676,0.090909,0.170732,1.000000,0.011364,0.139394,0.100000,0.097561,0.028986,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.014493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1681,0.002611,0.009524,0.000000,0.005848,0.000000,0.000000,0.003257,0.005780,0.004065,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000,0.000000


In [22]:
import joblib
cal_Jaccad = joblib.load("case/sim/Jac/ub/simJaccardFold1.joblib")
cal_Jaccad

,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
1,1.000000,0.035503,0.018750,0.013605,0.102439,0.155660,0.101266,0.031250,0.013793,0.106280,...,0.136029,0.041916,0.099206,0.060606,0.080000,0.039548,0.115207,0.046667,0.070000,0.130597
2,0.035503,1.000000,0.046154,0.038462,0.007692,0.094891,0.032653,0.014493,0.061224,0.038760,...,0.033816,0.161765,0.144654,0.230769,0.138462,0.098765,0.088889,0.087719,0.062500,0.034826
3,0.018750,0.046154,1.000000,0.076923,0.008475,0.045455,0.008368,0.054545,0.000000,0.025210,...,0.015075,0.015152,0.055901,0.062500,0.038168,0.026667,0.071429,0.041667,0.038835,0.005128
4,0.013605,0.038462,0.076923,1.000000,0.009615,0.000000,0.013393,0.047619,0.000000,0.009346,...,0.005348,0.019231,0.040000,0.058824,0.033898,0.016129,0.043103,0.090909,0.056818,0.011111
5,0.102439,0.007692,0.008475,0.009615,1.000000,0.086486,0.105455,0.052174,0.019802,0.051136,...,0.147186,0.031746,0.040179,0.015504,0.064171,0.029412,0.093923,0.036697,0.075949,0.146018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.039548,0.098765,0.026667,0.016129,0.029412,0.052980,0.031496,0.012821,0.000000,0.014184,...,0.032407,0.257143,0.136905,0.126582,0.226562,1.000000,0.033113,0.092308,0.024000,0.058537
940,0.115207,0.088889,0.071429,0.043103,0.093923,0.160428,0.134752,0.045802,0.025862,0.161850,...,0.190678,0.050360,0.116592,0.097015,0.091371,0.033113,1.000000,0.057377,0.141104,0.113360
941,0.046667,0.087719,0.041667,0.090909,0.036697,0.039370,0.008584,0.083333,0.062500,0.008696,...,0.015544,0.109091,0.108108,0.127273,0.120690,0.092308,0.057377,1.000000,0.041237,0.027027
942,0.070000,0.062500,0.038835,0.056818,0.075949,0.098837,0.073529,0.028302,0.022472,0.074534,...,0.119469,0.035088,0.057416,0.053097,0.056497,0.024000,0.141104,0.041237,1.000000,0.083333


In [46]:
# cal_Jaccad.iloc[942] 


In [19]:
cal_JaccadItem = joblib.load("case/sim/Jac/ib/simJaccardFold1.joblib")
print(f"sim {cal_JaccadItem.shape}")
cal_JaccadItem

sim (1682, 1682)


,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.000000,0.178744,0.150754,0.236607,0.091787,0.022843,0.416838,0.246637,0.302277,0.106796,...,0.002611,0.0,0.000000,0.000000,0.002611,0.0,0.0,0.0,0.002611,0.002611
2,0.178744,1.000000,0.139241,0.301887,0.175676,0.041667,0.187320,0.168067,0.103774,0.072289,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.009524,0.009524
3,0.150754,0.139241,1.000000,0.160377,0.090909,0.043956,0.147147,0.127273,0.150538,0.112782,...,0.000000,0.0,0.000000,0.000000,0.013333,0.0,0.0,0.0,0.000000,0.013333
4,0.236607,0.301887,0.160377,1.000000,0.170732,0.043716,0.257895,0.269373,0.208696,0.104072,...,0.000000,0.0,0.005848,0.005848,0.005848,0.0,0.0,0.0,0.005848,0.005848
5,0.091787,0.175676,0.090909,0.170732,1.000000,0.011364,0.139394,0.100000,0.097561,0.028986,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.014493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1681,0.002611,0.009524,0.000000,0.005848,0.000000,0.000000,0.003257,0.005780,0.004065,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000,0.000000


In [ ]:
# item_not_rated_by = np.where(item_ratings == 0)[0]
# print(f"item not rated by {len(item_not_rated_by)}")

    # print(f"pred unrated {PrediksiUnrated}")
    # Urutkan prediksi menurun dan ambil indeks top-N teratas
    # topNIndex = PrediksiUnrated.argsort()[::-1][:N] #sss, N teratas

## PREDIKSI

In [ ]:
#user_ratings != 0 bolean


In [ ]:
def TetanggaK(target_user, target_item, rating_matrix_np, similarity_np, k, jenis="user-based"):
    """
    Mengembalikan indeks K tetangga terdekat (user atau item) berdasarkan similarity.
    """
    rating_matrix = rating_matrix_np
    if jenis == "user-based":
        # Ambil semua rating terhadap item target, data rating untuk item target
        item_ratings = rating_matrix[:, target_item]
        print("item ratigs", item_ratings.shape)
        # User yang pernah memberi rating terhadap item tersebut (kecuali user target)
        # tetangga_idx = np.where((item_ratings != 0) & (np.arange(len(item_ratings)) != target_user))[0]
        # print(f"tetangga idx {tetangga_idx.shape}")
        tetangga_idx2 = np.where((item_ratings != 0))[0]
        print(f"tetangga idx2 {tetangga_idx2.shape}")
        npArg = np.arange(len([1,2,3,4])) 
        npArgFilter = npArg != 1
        print(f"np range1 {npArgFilter}")
        print(f"np range {npArg}")
        print(f"np range {npArg.shape}")
        # Ambil skor similarity
        sim_scores = similarity_np[target_user, tetangga_idx]
        # cek jumalh rating
        martiks = rating_matrix[target_user, tetangga_idx]
        print(f"matriks rating {martiks.shape}")

    else:  # item-based
        # Di item-based, user tetap user, item adalah target
        user_ratings = rating_matrix_np[target_user, :]
        # Cari item lain yang pernah diberi rating oleh user target (selain item target)
        # tetangga_idx = np.where((user_ratings != 0) & (np.arange(rating_matrix_np.shape[1]) != target_item))[0]
        tetangga_idx = np.where((user_ratings != 0))[0]
        print(f"tetangga idx {tetangga_idx.shape}")
        # Ambil skor similarity item-item
        sim_scores = similarity_np[tetangga_idx, target_item]
    # Urutkan tetangga berdasarkan similarity tertinggi
    if len(sim_scores) == 0:
        return []

    # print(f"sim scores {sim_scores}")
    print(f"sim scores {sim_scores.shape}")
    sorted_idx = np.argsort(-sim_scores)[:k]
    # print(f"sorted idx {sorted_idx}")
    # print(f"sim from soreted {sorted_idx} {sim_scores[sorted_idx]}")
    return tetangga_idx[sorted_idx]



In [ ]:
def PrediksiCF(RatingMatriks, similarityFunction, mean, meanCen, user, item, k=2, jenis="user-based"):
    """
    Menghitung prediksi rating menggunakan Collaborative Filtering berbasis user/item, versi optimal.
    """

    # Cache ke bentuk NumPy
    rating_np = RatingMatriks.to_numpy()

    # cek jumlah yang pernah memberikan rating 
    # print(f"jumlah rating item {item} {np.count_nonzero(rating_np[:, item])}")


    similarity_np = similarityFunction.to_numpy()
    print(similarity_np.shape)
    mean_np = mean.to_numpy()
    print(mean_np.shape)
    meanCen_np = meanCen.to_numpy()
    print("mean cen", meanCen_np.shape)

    # Set indeks untuk pencarian tetangga
    target_user = user
    target_item = item
    print(f"target user {target_user} {target_item}")
    mean_value = mean_np[user] if jenis == "user-based" else mean_np[item]
    print(f"mean {mean_value}")

    # Cari tetangga
    tetangga = TetanggaK(target_user, target_item, rating_np, similarity_np, k, jenis)
    print(f"tetangga {tetangga}")
    if len(tetangga) == 0:
        return float(mean_value)

    # Ambil similarity dan mean centered rating
    if jenis == "user-based":
        sim = similarity_np[user, tetangga] # ambil similarity user-user
        print(f'sim {sim}')
        mean_cen = meanCen_np[tetangga, item] 
        print(f"mean cen {mean_cen}")
        # cek tetangga dan item pada mean cen
        print(f"mean cen {meanCen_np[tetangga, item].shape}")
    else:  # item-based
        sim = similarity_np[tetangga, item] # ambil similarity item-item
        print(f"sim item {sim.shape}") 
        mean_cen = meanCen_np[user, tetangga] # ambil mean centered rating

    # vektor ((a1 * b1) + (a2 * b2))
    pembilang = np.dot(mean_cen, sim)
    print(f"pembilang {pembilang}")
    penyebut = np.sum(np.abs(sim))
    print(f"penyebut {penyebut}")

    prediksi = mean_value + (pembilang / penyebut) if penyebut != 0 else mean_value
    print(f"prediksi {mean_value} + {pembilang} / {penyebut}")
    return float(prediksi)

In [24]:
loadHybridJac1 = joblib.load("case/prediksiModel/Jac/ub/1/5.joblib")
loadHybridJac1

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.000000,3.000000,4.000000,3.000000,3.000000,3.218691,4.000000,1.000000,5.000000,3.728815,...,2.534423,3.544426,3.222151,2.222151,3.469500,1.559986,3.559986,2.559986,3.700818,3.315551
2,4.000000,3.222722,2.963838,3.247616,3.891988,4.830095,3.889122,4.532790,4.440090,2.000000,...,2.652941,3.662944,3.340670,2.340670,3.588018,1.678505,3.678505,2.678505,3.819337,3.434069
3,3.814737,1.785034,2.182020,3.027970,3.085057,2.371788,3.193913,3.120864,3.799709,3.388455,...,1.852941,2.862944,2.540670,1.540670,2.788018,0.878505,2.878505,1.878505,3.019337,2.634069
4,5.046868,3.067127,4.001402,3.844595,3.888865,4.428490,4.923513,4.586607,4.893147,4.506614,...,3.210084,4.220087,3.897813,2.897813,4.145161,2.235648,4.235648,3.235648,4.376480,3.991212
5,3.750135,2.815731,2.322002,2.763512,3.070289,2.139646,3.667401,3.813842,2.751794,3.071424,...,1.808985,2.818988,2.496714,1.496714,2.744062,2.956044,2.956044,2.956044,2.975381,2.590113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.776029,3.958011,4.049760,3.642780,4.181584,5.034501,3.625027,4.829840,5.000000,4.435371,...,3.118247,4.128250,3.805976,2.805976,4.053325,2.143811,4.143811,3.143811,4.284643,3.899376
940,3.403160,3.252267,2.487956,2.000000,2.443996,2.845468,4.000000,5.000000,3.000000,3.779176,...,2.310885,3.320888,2.998614,1.998614,3.245962,1.336449,3.336449,2.336449,3.477281,3.092013
941,5.000000,3.576549,3.975729,2.953050,3.741924,4.123861,4.000000,4.972952,3.971818,3.854685,...,2.898396,3.908399,3.586124,2.586124,3.833473,1.923959,3.923959,2.923959,4.064792,3.679524
942,4.878181,3.814061,3.478054,4.069125,4.053574,4.071380,4.239001,5.162838,4.500917,4.627949,...,3.118764,4.128767,3.806493,2.806493,4.053841,2.144327,4.144327,3.144327,4.285160,3.899892


In [23]:
# user based [1, 16]

PrediksiUserBased = PrediksiCF(rating_Matriks, cal_Jaccad, cal_MeanRatingUser, cal_MeanCenteredUser, 1, 16, 5, "user-based")
PrediksiUserBased

(943, 943)
(943, 1)
mean cen (943, 1682)
target user 1 16
mean [3.8]
item ratigs (943,)
tetangga idx (72,)
tetangga idx2 (72,)
np range1 [ True False  True  True]
np range [0 1 2 3]
np range (4,)
matriks rating (72,)
sim scores (72,)
tetangga [547 870 446 454 296]
sim [0.16071429 0.09929078 0.09868421 0.08737864 0.08130081]
mean cen [-0.67096774 -0.49565217 -0.57480315 -0.5        -0.40860215]
mean cen (5,)
pembilang -0.2906807948601824
penyebut 0.5273687301672747
prediksi [3.8] + -0.2906807948601824 / 0.5273687301672747


3.248809195099637

In [30]:
# item based [1, 16] 272 283 281 287  99

PrediksiItemBased = PrediksiCF(rating_Matriks, cal_JaccadItem, cal_MeanRatingItem, cal_MeanCenteredItem, 1, 16, 5, "item-based")
PrediksiItemBased

(1682, 1682)
(1682, 1)
mean cen (943, 1682)
target user 1 16
mean [3.18055556]
tetangga idx (40,)
sim scores (40,)
tetangga [272 283 281 287  99]
sim item (5,)
pembilang 0.2740204007003504
penyebut 0.6528737293548885
prediksi [3.18055556] + 0.2740204007003504 / 0.6528737293548885


3.600269794894168

In [28]:
# # maksimal rating pada load hybrid
# maxLoadHybrid = loadHybridJac1.max()
# print(f"Max rating : {maxLoadHybrid}")

In [ ]:
# item based
PrediksiItemBased = PrediksiCF(rating_Matriks, cal_JaccadItem, cal_MeanRatingItem, cal_MeanCenteredItem, 0, 9, 40, "item-based")
PrediksiItemBased

3.8805137869975996

In [ ]:
def prediksi_semua_matriks2(RatingMatriks, similarityFunction, mean, meanCen, k=2, jenis="user-based"):
    """
    Mengisi semua rating kosong dalam matriks dengan hasil prediksi CF.
    """
    # salin rating matriks
    hasil_matriks = RatingMatriks.copy()
    # mendapatkan ukuran matriks rating/ ambil
    num_users, num_items = hasil_matriks.shape
    # loop setiap user
    for user in range(num_users):
        # loop setiap item
        for item in range(num_items):
            if hasil_matriks.iloc[user, item] == 0:
                pred_rating = PrediksiCF(
                    RatingMatriks=RatingMatriks,
                    similarityFunction=similarityFunction,
                    mean=mean,
                    meanCen=meanCen,
                    user=user,
                    item=item,
                    k=k,
                    jenis=jenis # user-based atau item-based
                )
                # isi prediksi hasil ke dalam matriks
                hasil_matriks.iloc[user, item] = pred_rating
    return hasil_matriks

In [ ]:
calHasilPrediksi = joblib.load("case/prediksiModel/Jac/ub/1/5.joblib")
print(calHasilPrediksi.shape)

In [ ]:
def simpan_semua_prediksi_k(
    RatingMatriks, similarityFunction, mean, meanCen,
    variasi_k, jenis="user-based",
    folder_output="case/prediksi/Jac/ub/5"
):
    for k in variasi_k:
        # Hitung prediksi
        prediksi = prediksi_semua_matriks2(
            RatingMatriks=RatingMatriks,
            similarityFunction=similarityFunction,
            mean=mean,
            meanCen=meanCen,
            k=k,
            jenis=jenis
        )
        # Siapkan path file
        path_file = os.path.join(folder_output, f"{k}.joblib")
        os.makedirs(folder_output, exist_ok=True)

        # Simpan hasil prediksi
        joblib.dump(prediksi, path_file)
        print(f"✔️ Prediksi untuk k={k} disimpan di: {path_file}")

In [ ]:
simpan_semua_prediksi_k(
    RatingMatriks=rating_Matriks,
    similarityFunction=cal_Jaccad,
    mean=cal_MeanRatingUser,
    meanCen=cal_MeanCenteredUser,
    variasi_k=[5, 10, 15, 18, 20, 25, 30, 40, 50, 100, 200],
    jenis="user-based",
    folder_output="case/prediksi/Jac/ub/1"
)

NameError: name 'simpan_semua_prediksi_k' is not defined